## 第07章 函数装饰器和闭包

### 7.1 装饰器基本知识

- 装饰器是可调用对象，其参数是另一个函数
- 假如有个名为decorate的装饰器

```python
@decorate
def target():
  print('running target()')

# 等价于
def target():
  print('running target()')
target = decorate(target)
```

In [1]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

@deco
def target():
  print('running target()')

In [3]:
target(), target

running inner()


(None, <function __main__.deco.<locals>.inner()>)

- 严格来说，装饰器只是语法糖
- 装饰器的一大特性是把被装饰的函数替换为其他函数(e.g. `target()`->`inner()`)

### 7.2 Python何时执行装饰器
- 被装饰函数定义后立即执行，通常是加载的时候

In [6]:
registry = []

def register(func):
  print(f'running register({func})')
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

running register(<function f1 at 0x0000017B637EB880>)
running register(<function f2 at 0x0000017B6383BEC0>)


### 7.4 变量作用域规则

- python判断`b`是局部变量，所以报错

In [8]:
b = 6
def f1():
  print(b)
  b = 9
f1()

UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [9]:
b = 6
def f1():
  global b
  print(b)
  b = 9
f1()

6


### 7.5 闭包
- 闭包指延伸了作用域的函数，能访问定义域之外的非全局变量
- 与匿名函数容易弄混

In [10]:
def make_averager():
  series = []

  def averager(val):
    series.append(val)
    return sum(series) / len(series)
  return averager
avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

- 在`averager`中，series是自由变量。
  - free variable: 指未在本地作用域内绑定的变量

### 7.6 nonlocal声明

In [12]:
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    count += 1
    total += new_value
    return total / count
  return averager
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

- count 是数字或任何不可变类型。当我们赋值的时候会把count变成局部变量
- `nonlocal`将某个变量声明为自由变量

In [13]:
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  return averager
avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

### 7.7 实现一个简单的装饰器

In [48]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args) 
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [49]:
@clock
def factorial(n):
  '''Return n!'''
  return 1 if n < 2 else n * factorial(n - 1)

factorial(6)

[0.00000060s] factorial(1) -> 1
[0.00013600s] factorial(2) -> 2
[0.00015160s] factorial(3) -> 6
[0.00015810s] factorial(4) -> 24
[0.00016350s] factorial(5) -> 120
[0.00016990s] factorial(6) -> 720


720

In [50]:
factorial.__name__, factorial.__doc__

('clocked', None)

- 这个简单的装饰器遮蔽了`__name__` `__doc__`
- 并且其不能处理关键字参数
- 使用`functools.wrap`把相关属性从func复制到clocked

In [59]:
import time
import functools
def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.perf_counter()
    result = func(*args, **kwargs)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_lst = []
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = [f'{k}={w}' for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result} ')
    return result
  return clocked

In [60]:
@clock
def factorial(n):
  '''Return n!'''
  return 1 if n < 2 else n * factorial(n - 1)

factorial(6)

[0.00000080s] factorial(1) -> 1 
[0.00021190s] factorial(2) -> 2 
[0.00024190s] factorial(3) -> 6 
[0.00025480s] factorial(4) -> 24 
[0.00026670s] factorial(5) -> 120 
[0.00027900s] factorial(6) -> 720 


720

In [23]:
factorial.__name__, factorial.__doc__

('factorial', 'Return n!')

### 7.8 标准库中的装饰器

#### 7.8.1 使用functools.lru_cache做备忘
- 缓存键要是可散列的，因为内部使用字典
- 接受maxsize参数, max_size应该是2的幂

In [68]:
@clock
def fibonacci(n):
  if n < 2:
    return n
  return fibonacci(n - 2) + fibonacci(n - 1)
fibonacci(6)

[0.00000040s] fibonacci(0) -> 0 
[0.00000040s] fibonacci(1) -> 1 
[0.00006100s] fibonacci(2) -> 1 
[0.00000020s] fibonacci(1) -> 1 
[0.00000030s] fibonacci(0) -> 0 
[0.00000030s] fibonacci(1) -> 1 
[0.00000990s] fibonacci(2) -> 1 
[0.00001910s] fibonacci(3) -> 2 
[0.00008910s] fibonacci(4) -> 3 
[0.00000020s] fibonacci(1) -> 1 
[0.00000020s] fibonacci(0) -> 0 
[0.00000020s] fibonacci(1) -> 1 
[0.00000870s] fibonacci(2) -> 1 
[0.00002030s] fibonacci(3) -> 2 
[0.00000020s] fibonacci(0) -> 0 
[0.00000020s] fibonacci(1) -> 1 
[0.00000870s] fibonacci(2) -> 1 
[0.00000020s] fibonacci(1) -> 1 
[0.00000020s] fibonacci(0) -> 0 
[0.00000020s] fibonacci(1) -> 1 
[0.00000880s] fibonacci(2) -> 1 
[0.00001760s] fibonacci(3) -> 2 
[0.00003480s] fibonacci(4) -> 3 
[0.00006350s] fibonacci(5) -> 5 
[0.00016190s] fibonacci(6) -> 8 


8

In [69]:
@functools.lru_cache(maxsize=128)
@clock
def fibonacci(n):
  if n < 2:
    return n
  return fibonacci(n - 2) + fibonacci(n - 1)
fibonacci(6)

[0.00000060s] fibonacci(0) -> 0 
[0.00000040s] fibonacci(1) -> 1 
[0.00005740s] fibonacci(2) -> 1 
[0.00000050s] fibonacci(3) -> 2 
[0.00006760s] fibonacci(4) -> 3 
[0.00000040s] fibonacci(5) -> 5 
[0.00007860s] fibonacci(6) -> 8 


8